In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os
import pickle
import sklearn
import sys

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, Normalizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier,NearestNeighbors
from sklearn.model_selection import GridSearchCV

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [3]:
%aimport data.load_delicious
%aimport features.build_features
%aimport helpers.files,helpers.labels

In [4]:
from data.load_delicious import load_taginfo_into_dataframe
from helpers.files import get_directory_name_from_hash
from helpers.labels import truncate_labels
from features.preprocess import clean_text_delicious

In [5]:
ROOT = "/media/felipe/SAMSUNG/delicious/delicioust140/"
TAGINFO = ROOT+"taginfo.xml"
INTERIM_DATA_ROOT = "../data/interim/delicious-t140/"
MAX_NB_WORDS = 2000
LABELS_MIN_DOC_FRACTION = 0.01

In [6]:
if os.path.isfile(INTERIM_DATA_ROOT+"docs_df.p"):
    docs_df = pickle.load( open( INTERIM_DATA_ROOT+"docs_df.p", "rb" ))
else:
    docs_df = load_taginfo_into_dataframe(TAGINFO)
    pickle.dump(docs_df,open(INTERIM_DATA_ROOT+"docs_df.p","wb"))

In [7]:
num_documents = len(docs_df)
num_documents

143716

In [8]:
docs_df.head(10)

,filename,filetype,hash,tags,url,num_users,num_tags
0,66fa11b33b6ac183314892703c20fa47.html,html,66fa11b33b6ac183314892703c20fa47,"python,programming,standards,style,coding,refe...",http://lists.osafoundation.org/pipermail/dev/2...,27,8
1,21899d001299ceadc852ed22e1b2b725.html,html,21899d001299ceadc852ed22e1b2b725,"compiler,programming,llvm,vm,development,compi...",http://llvm.org/,830,25
2,bd7c9734cd1a5c8a55328a1a9ce4d4d2.html,html,bd7c9734cd1a5c8a55328a1a9ce4d4d2,"linux,software,opensource,ubuntu,windows,alter...",http://linuxappfinder.com/alternatives,47,19
3,ff186471d34e1440845b80d61733f8ef.html,html,ff186471d34e1440845b80d61733f8ef,"tomboy,gnome,linux,software,wiki,notes",http://live.gnome.org/Tomboy,7,6
4,cc7afd7b1b9e0c29ba72978b5edd8ed5.html,html,cc7afd7b1b9e0c29ba72978b5edd8ed5,"blogs,teaching,web2.0",http://lisahistory.net/wordpress/,7,3
5,e9466d13558200b7b084fae5d0d81b57.html,html,e9466d13558200b7b084fae5d0d81b57,"books,literature,search,recommendations,refere...",http://literature-map.com/,373,25
6,2e14957206bad2bdf5fb29564c6c863c.html,html,2e14957206bad2bdf5fb29564c6c863c,"deviantart,photography,inspiration",http://littlemewhatever.deviantart.com/,15,3
7,e84764f03cc25dcf50207bfa534a9e84.html,html,e84764f03cc25dcf50207bfa534a9e84,"phone,mobile,cellphone,sprint,cell,apps,treo,t...",http://livemobile.blogspot.com/,55,13
8,a6f8b9ccfe1219ab79be4001c08b31ab.html,html,a6f8b9ccfe1219ab79be4001c08b31ab,"video,videos,bizarre,weird,movies,lists,cool,s...",http://listverse.com/bizarre/top-10-most-bizar...,64,15
9,72c4c07a25937957f599c7524f94cd75.html,html,72c4c07a25937957f599c7524f94cd75,"movies,film,cool,lists",http://listverse.com/entertainment/top-15-indi...,8,4


In [9]:
# TODO optimize this because currently this does one I/O OP per loop
def load_contents(hash):
    file_path = ROOT+"fdocuments/"+get_directory_name_from_hash(hash)+"/"+hash+".html"
       
    with open(file_path,"r",encoding='utf-8', errors='ignore') as f:
        contents = f.read()
        
    return contents

In [10]:
%%time

if os.path.isfile(INTERIM_DATA_ROOT+"sample_df.p"):
    sample_df = pickle.load(open(INTERIM_DATA_ROOT+"sample_df.p", "rb"))
else:
    random_indices = np.random.choice(docs_df.index.values, int(num_documents/50), replace=False)
    sample_df = docs_df.loc[random_indices]
    sample_df = sample_df.reset_index().drop(['index'],axis=1)
    sample_df['contents'] = sample_df['hash'].map(lambda hash: load_contents(hash))
    pickle.dump(sample_df,open(INTERIM_DATA_ROOT+"sample_df.p","wb"))

CPU times: user 84 ms, sys: 56 ms, total: 140 ms
Wall time: 140 ms


In [11]:
num_documents = len(sample_df)
num_documents

2874

In [12]:
sample_df.head(10)

,filename,filetype,hash,tags,url,num_users,num_tags,contents
0,37296e86ac599d7fe9aaff17a3037400.html,html,37296e86ac599d7fe9aaff17a3037400,"flash,air,adobe,blog,flex,design,webdesign,act...",http://www.webkitchen.be/,65,23,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
1,0b294ed63a75b9d264af303f5c5eeea9.html,html,0b294ed63a75b9d264af303f5c5eeea9,"french,language,franÃ§ais,resources,education,...",http://www.uni.edu/becker/french31.html,123,21,<HTML>\n<HEAD>\n\n<TITLE>Best French Websites<...
2,5d7ec003095ae0f3123b2e3da4b3fb95.html,html,5d7ec003095ae0f3123b2e3da4b3fb95,"music,mp3,history,free,audio,downloads,songs,d...",http://www.foldedspace.org/weblog/2006/06/in_t...,566,25,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
3,8ca4efc75f6399dfe7e4bffbfc0c08d4.html,html,8ca4efc75f6399dfe7e4bffbfc0c08d4,"ssh,sftp,security,linux,database,backup,howto,...",http://ask-leo.com/how_can_i_automate_an_sftp_...,21,10,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
4,ce9a31b508d0505b0e4cb2bbf4fe3d55.html,html,ce9a31b508d0505b0e4cb2bbf4fe3d55,"ajax,rails,tutorial",http://railsonedge.blogspot.com/2008/03/tutori...,7,3,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 S..."
5,1bcbe8e8838244aca41bfbe69a958f24.html,html,1bcbe8e8838244aca41bfbe69a958f24,"silverlight,charts,opensource,chart,.net,free,...",http://www.visifire.com/silverlight_charts_gal...,47,15,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
6,8384c9cae634c9dff115a741926e0fc9.html,html,8384c9cae634c9dff115a741926e0fc9,"parenting,education,baby,children,learning,lan...","http://www.time.com/time/health/article/0,8599...",121,25,<!--[if IE 5]> Vignette StoryServer 5.0 Fri Ma...
7,2ad1b840589b35378c8757739a461b1d.html,html,2ad1b840589b35378c8757739a461b1d,"language,dictionary,swear,fun,reference,funny,...",http://www.youswear.com/index.asp,18,14,
8,78f632abdab2116c806b8ab81029fce7.html,html,78f632abdab2116c806b8ab81029fce7,"history,image,compression,lenna,fun,internet,i...",http://www.cs.cmu.edu/~chuck/lennapg/,58,25,<html>\n\n<head>\n<title>The Rest of the Lenna...
9,8c1fd30e1547407eab96254252d433d4.html,html,8c1fd30e1547407eab96254252d433d4,"science,space,astronomy",http://dvice.com/archives/2008/06/astronomers_...,4,3,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."


In [13]:
tag_sets = sample_df["tags"].values

all_tags = set()

for tag_set in tag_sets:
    for tag in tag_set.split(','):
        all_tags.add(tag)

In [14]:
len(all_tags)

7201

In [15]:
min_nb_docs = int(len(sample_df)* LABELS_MIN_DOC_FRACTION)
min_nb_docs

28

In [16]:
sample_df["tags_split"] = sample_df["tags"].map(lambda tagstring: tagstring.split(","))
labels = sample_df["tags_split"].values

In [17]:
truncated_labels = truncate_labels(labels,min_nb_docs)

In [18]:
mlb = MultiLabelBinarizer()
binary_label_data = mlb.fit_transform(truncated_labels)

In [19]:
binary_label_data.shape

(2874, 242)

In [20]:
data = sample_df["contents"].values

In [21]:
data[0][:1000]

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head profile="http://gmpg.org/xfn/11">\n<title>Serge Jespers</title>\n<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" />\n<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" />\n<meta http-equiv="pragma" content="no-cache" />\n<meta http-equiv="cache-control" content="no-cache" />\n<link rel="stylesheet" type="text/css" href="http://www.webkitchen.be/wp-content/themes/smashingtheme/style.css" />\n<script type="text/javascript" src="http://www.webkitchen.be/wp-content/themes/smashingtheme/javascript/imghover.js"> </script>\n<link rel="alternate" type="application/rss+xml" title="RSS 2.0" href="http://www.webkitchen.be/feed/" />\n<link rel="alternate" type="text/xml" title="RSS .92" href="http://www.webkitchen.be/feed/rss/" />\n<link rel="alternate" type="application/atom+xml" title

In [22]:
pipeline = Pipeline([
    ('vect', CountVectorizer(preprocessor=clean_text_delicious, max_features=MAX_NB_WORDS)),
    ('tf',TfidfTransformer()),
    ('lda',LatentDirichletAllocation()),
    ('norm',Normalizer()),
    ('clf', OneVsRestClassifier(SVC(),n_jobs=-1)),
])

In [25]:
# use later
parameters = [
    {
        "tf__use_idf":[False],
        "tf__norm":['l1','l2',None],
        "lda__n_components":[2,5,10,20],
        "lda__max_iter":[5,10],
        "lda__learning_method":["online"],
        "norm__norm":["l2"],
        "clf__estimator__C":[1,10,100,100],
        "clf__estimator__gamma":[0.1,0.01,0.001],
        "clf__estimator__kernel": ["rbf"]
    },
    {
        "tf__use_idf":[False],
        "tf__norm":['l1','l2',None],
        "lda__n_components":[2,5,10,20],
        "lda__max_iter":[5,10],
        "lda__learning_method":["online"],
        "norm__norm":["l2"],
        "clf__estimator__C":[1,10,100,100],
        "clf__estimator__kernel": ["linear"]
    },    
]

In [26]:
%time

# start with minus infinity as your
# current best_score
best_score = float("-inf")

for g in ParameterGrid(parameters):
    pipeline.set_params(**g)
    
    X_train, X_validation, y_train, y_validation = train_test_split(data, binary_label_data, test_size = 0.25)
    
    # here you call fit with whatever data you want
    pipeline.fit(X_train,y_train)
    
    # again, choose the validation data 
    # yourself
    y_pred_train = pipeline.predict(X_train)    
    y_pred_validation = pipeline.predict(X_validation)
    
    # I've used f1-score as an example, but you can use
    # any metric you want.
    train_score = f1_score(y_train,y_pred_train, average='micro')
    val_score = f1_score(y_validation,y_pred_validation, average='micro')
    
    current_score = val_score
    
    # show results
    print("training F1: {}".format(train_score))
    print("validation F1: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    # update the best_score if needed
    if current_score > best_score:
        best_score = current_score
        best_grid = g

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


training F1: 0.0
validation F1: 0.0
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 2, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': 'l1', 'clf__estimator__C': 1}



/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


training F1: 0.0
validation F1: 0.0
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 2, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': 'l2', 'clf__estimator__C': 1}



...

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


training F1: 0.0
validation F1: 0.0
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 2, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': 'l2', 'clf__estimator__C': 100}



/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


training F1: 0.0
validation F1: 0.0
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 2, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': None, 'clf__estimator__C': 100}

training F1: 0.005812918915712675
validation F1: 0.003516174402250352
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 5, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': 'l1', 'clf__estimator__C': 100}

training F1: 0.002599550986647761
validation F1: 0.0
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_idf': False, 'lda__learning_method': 'online', 'lda__n_components': 5, 'clf__estimator__kernel': 'rbf', 'lda__max_iter': 5, 'tf__norm': 'l2', 'clf__estimator__C': 100}

training F1: 0.008246451080874125
validation F1: 0.007094714437743881
grid: {'norm__norm': 'l2', 'clf__estimator__gamma': 0.1, 'tf__use_

KeyboardInterrupt: 